# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./data/final_train_data_ver2.csv")
test_data = pd.read_csv("./data/final_test_data_ver2.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result_encoded',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'time_gap_All'
]

In [4]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [10]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [11]:
# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveRandomForestClassifier(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 800, 3000),
        'max_depth' : trial.suggest_int('max_depth', 20, 100),
        'min_samples_split' : trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 8),
        'criterion' : trial.suggest_categorical("criterion", ["entropy",]),
        'class_weight' : trial.suggest_categorical("class_weight", ["balanced"]),
        'random_state': RANDOM_STATE
    }
    
    model = RandomForestClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_autoclave.drop("target", axis=1),
    train_data_autoclave["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveRandomForestClassifier(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

/tmp/ipykernel_331/3435388659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})
[I 2024-08-28 08:53:57,887] A new study created in memory with name: no-name-85650887-1709-46ee-b125-8304e60cb734
[I 2024-08-28 08:54:21,616] Trial 0 finished with value: 0.17554324669791221 and parameters: {'n_estimators': 1055, 'max_depth': 73, 'min_samples_split': 9, 'min_samples_leaf': 5, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.17554324669791221.
[I 2024-08-28 08:55:11,887] Trial 1 finished with value: 0.16732495511669657 and parameters: {'n_estimators': 2295, 'max_depth': 75, 'min_samples_split': 20, 'min_samples_leaf': 6,

[I 2024-08-28 09:19:32,800] Trial 28 finished with value: 0.1918238993710692 and parameters: {'n_estimators': 2381, 'max_depth': 93, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 28 with value: 0.1918238993710692.
[I 2024-08-28 09:20:19,246] Trial 29 finished with value: 0.17880085653104924 and parameters: {'n_estimators': 1995, 'max_depth': 89, 'min_samples_split': 2, 'min_samples_leaf': 3, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 28 with value: 0.1918238993710692.
[I 2024-08-28 09:21:09,982] Trial 30 finished with value: 0.18742138364779878 and parameters: {'n_estimators': 2110, 'max_depth': 62, 'min_samples_split': 3, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 28 with value: 0.1918238993710692.
[I 2024-08-28 09:22:18,903] Trial 31 finished with value: 0.1891891891891892 and parameters: {'n_estimators': 2805, 'max_depth': 95, 'min_samples_split'

[I 2024-08-28 09:48:19,737] Trial 57 finished with value: 0.18566392479435959 and parameters: {'n_estimators': 2883, 'max_depth': 83, 'min_samples_split': 6, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 35 with value: 0.19383921863260706.
[I 2024-08-28 09:49:27,974] Trial 58 finished with value: 0.1899070385126162 and parameters: {'n_estimators': 2815, 'max_depth': 86, 'min_samples_split': 5, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 35 with value: 0.19383921863260706.
[I 2024-08-28 09:50:30,863] Trial 59 finished with value: 0.19383921863260706 and parameters: {'n_estimators': 2543, 'max_depth': 90, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 35 with value: 0.19383921863260706.
[I 2024-08-28 09:51:27,275] Trial 60 finished with value: 0.17380457380457381 and parameters: {'n_estimators': 2510, 'max_depth': 77, 'min_samples_sp

[I 2024-08-28 10:17:38,440] Trial 86 finished with value: 0.1674008810572687 and parameters: {'n_estimators': 2755, 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 7, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:18:52,977] Trial 87 finished with value: 0.1918238993710692 and parameters: {'n_estimators': 2983, 'max_depth': 63, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:19:54,839] Trial 88 finished with value: 0.19088319088319092 and parameters: {'n_estimators': 2510, 'max_depth': 52, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:20:36,424] Trial 89 finished with value: 0.18414918414918416 and parameters: {'n_estimators': 1758, 'max_depth': 30, 'min_samples_spl

[I 2024-08-28 10:46:43,036] Trial 115 finished with value: 0.1924447612259444 and parameters: {'n_estimators': 2438, 'max_depth': 87, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:47:38,421] Trial 116 finished with value: 0.19369369369369369 and parameters: {'n_estimators': 2250, 'max_depth': 92, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:48:35,794] Trial 117 finished with value: 0.18706842435655993 and parameters: {'n_estimators': 2387, 'max_depth': 41, 'min_samples_split': 3, 'min_samples_leaf': 2, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 10:49:31,548] Trial 118 finished with value: 0.1907473309608541 and parameters: {'n_estimators': 2275, 'max_depth': 92, 'min_samples

[I 2024-08-28 11:16:43,619] Trial 144 finished with value: 0.1921708185053381 and parameters: {'n_estimators': 2453, 'max_depth': 91, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:17:44,833] Trial 145 finished with value: 0.19383921863260706 and parameters: {'n_estimators': 2490, 'max_depth': 88, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:18:48,504] Trial 146 finished with value: 0.19369369369369369 and parameters: {'n_estimators': 2583, 'max_depth': 94, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:19:59,066] Trial 147 finished with value: 0.19070133963750985 and parameters: {'n_estimators': 2504, 'max_depth': 87, 'min_sample

[I 2024-08-28 11:47:53,189] Trial 173 finished with value: 0.19383921863260706 and parameters: {'n_estimators': 2479, 'max_depth': 100, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:48:56,591] Trial 174 finished with value: 0.1904761904761905 and parameters: {'n_estimators': 2600, 'max_depth': 96, 'min_samples_split': 4, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:49:56,872] Trial 175 finished with value: 0.1904012588512982 and parameters: {'n_estimators': 2438, 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 11:51:01,818] Trial 176 finished with value: 0.19354838709677422 and parameters: {'n_estimators': 2640, 'max_depth': 94, 'min_sample

[I 2024-08-28 12:16:08,101] Trial 202 finished with value: 0.19383921863260706 and parameters: {'n_estimators': 2536, 'max_depth': 100, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 12:17:10,041] Trial 203 finished with value: 0.19369369369369369 and parameters: {'n_estimators': 2494, 'max_depth': 98, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 12:18:14,417] Trial 204 finished with value: 0.19369369369369369 and parameters: {'n_estimators': 2437, 'max_depth': 95, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.
[I 2024-08-28 12:19:41,657] Trial 205 finished with value: 0.1919748229740362 and parameters: {'n_estimators': 2596, 'max_depth': 89, 'min_sampl

KeyboardInterrupt: 

[I 2024-08-28 10:13:25,153] Trial 82 finished with value: 0.19410496046010065 and parameters: {'n_estimators': 2744, 'max_depth': 27, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 82 with value: 0.19410496046010065.